In [1]:
!pip install graphviz==0.20

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 2.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
dados = pd.read_csv(uri)
dados.head()

ModuleNotFoundError: No module named 'pandas'

In [ ]:
a_renomear = {
    
    "mileage_per_year":"milhas_por_ano",
    "model_year":"modelo_ano",
    "price":"preco",
    "sold":"vendido"
}

dados = dados.rename(columns = a_renomear)
dados.head()

,Unnamed: 0,milhas_por_ano,modelo_ano,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [ ]:
#Vendidos (features):
#0 -> não
#1 -> sim

a_trocar = {
    'yes':1,
    'no':0
}

dados.vendido = dados.vendido.map(a_trocar)#mapeando o dicionario troca. 
dados.head()

,Unnamed: 0,milhas_por_ano,modelo_ano,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


In [ ]:
from datetime import datetime

ano_atual = datetime.today().year #extraindo ano.
dados['idade_modelo'] =  ano_atual - dados.modelo_ano  #Criando mais uma coluna com a idade do veículo.
dados.head()


,Unnamed: 0,milhas_por_ano,modelo_ano,preco,vendido,idade_modelo
0,0,21801,2000,30941.02,1,22
1,1,7843,1998,40557.96,1,24
2,2,7109,2006,89627.50,0,16
3,3,26823,2015,95276.14,0,7
4,4,7935,2014,117384.68,1,8


In [ ]:
dados ['km_por_ano'] = dados.milhas_por_ano * 1.60934
dados.head()

,Unnamed: 0,milhas_por_ano,modelo_ano,preco,vendido,idade_modelo,km_por_ano
0,0,21801,2000,30941.02,1,22,35085.22134
1,1,7843,1998,40557.96,1,24,12622.05362
2,2,7109,2006,89627.50,0,16,11440.79806
3,3,26823,2015,95276.14,0,7,43167.32682
4,4,7935,2014,117384.68,1,8,12770.11290


In [ ]:
dados = dados.drop(columns = ['Unnamed: 0', 'milhas_por_ano', 'modelo_ano'], axis=1) #Se não for "setado" por padrão o comando drop exclui uma linha da tabela. Para o proposito desse projeto, precisamos excluir colunas ("axis(eixo)=1") -. Excluindo colunas. . 
dados.head()



,preco,vendido,idade_modelo,km_por_ano
0,30941.02,1,22,35085.22134
1,40557.96,1,24,12622.05362
2,89627.50,0,16,11440.79806
3,95276.14,0,7,43167.32682
4,117384.68,1,8,12770.11290


In [ ]:
x = dados[["preco", "idade_modelo", "km_por_ano"]]
y = dados["vendido"]


In [ ]:
from sklearn.model_selection import train_test_split #Separando o treino e teste de um conjunto de dados qualquer
from sklearn.svm import LinearSVC #estimador
from sklearn.metrics import accuracy_score
import numpy as np


SEED = 5  #Definindo a ordem dos números aleatórios
np.random.seed(SEED)
#random_stateint, RandomState instance or None, default=None -> Controls the shuffling applied to the data before applying the split. Pass an int for reproducible output across multiple function calls.
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.2,stratify = y) #Separando os dados para treino e teste e estipulando o tamanho para o teste, também. 
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x),len(teste_x)))

modelo = LinearSVC()
modelo.fit(treino_x, treino_y)

previsoes = modelo.predict(teste_x) #Predizendo através dos meus dados de teste, após ser treinado com meus dados de treino
taxa_de_acerto = accuracy_score(teste_y, previsoes)* 100 #Imprimir minha taxa de acerto ou "acurácia" - Comparação com os meus dados reais de teste
print("A acurácia foi %.2f" % taxa_de_acerto) 

Treinaremos com 8000 elementos e testaremos com 2000 elementos
A acurácia foi 54.25


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [ ]:
from sklearn.dummy import DummyClassifier #Estimador "bobo" para eu ter minha baseline e comprar com o meu modelo.
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) #Return the mean accuracy on the given test data and labels.

print("A acurácia do dummy stratified foi %.2f" % acuracia) 

A acurácia do dummy stratified foi 0.58


In [ ]:
from sklearn.dummy import DummyClassifier #Estimador "bobo" para eu ter minha baseline e comprar com o meu modelo.
dummy_mostfrequent = DummyClassifier()
dummy_mostfrequent.fit(treino_x, treino_y)
previsoes = dummy_mostfrequent.predict(teste_x)

taxa_de_acerto = accuracy_score(teste_y, previsoes)* 100 #Imprimir minha taxa de acerto ou "acurácia" - Comparação com os meus dados reais de teste
print("A acurácia do dummy stratified foi %.2f" % taxa_de_acerto) 

A acurácia do dummy stratified foi 58.00


In [ ]:
from sklearn.model_selection import train_test_split #Separando o treino e teste de um conjunto de dados qualquer
from sklearn.svm import SVC #estimador
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import StandardScaler

SEED = 5  #Definindo a ordem dos números aleatórios
np.random.seed(SEED)
#random_stateint, RandomState instance or None, default=None -> Controls the shuffling applied to the data before applying the split. Pass an int for reproducible output across multiple function calls.
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,stratify = y) #Separando os dados para treino e teste e estipulando o tamanho para o teste, também. 
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x),len(teste_x)))

#Trinando o escalador - Treinando um novo processo de escala: Isso o ocorre devido a discrepância entre os eixos: em X, teremos valores de 0 a 100, e em Y de 0 a 30000. Esses algoritmos são muito suscetíveis a escala, e darão menos valor para variações menores, como é o caso de X.
scaler = StandardScaler() #Escalas distintas em features causam um desbalanço no algoritmo
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = SVC()
modelo.fit(treino_x, treino_y)

previsoes = modelo.predict(teste_x) #Predizendo através dos meus dados de teste, após ser treinado com meus dados de treino
taxa_de_acerto = accuracy_score(teste_y, previsoes)* 100 #Imprimir minha taxa de acerto ou "acurácia" - Comparação com os meus dados reais de teste
print("A acurácia foi %.2f" % taxa_de_acerto) 

Treinaremos com 8000 elementos e testaremos com 2000 elementos
A acurácia foi 77.48


**Usando o decision tree classifier**

In [ ]:
from sklearn.model_selection import train_test_split #Separando o treino e teste de um conjunto de dados qualquer
from sklearn.tree import DecisionTreeClassifier #estimador
from sklearn.tree import export_graphviz
import graphviz
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import StandardScaler

SEED = 5  #Definindo a ordem dos números aleatórios
np.random.seed(SEED)
#random_stateint, RandomState instance or None, default=None -> Controls the shuffling applied to the data before applying the split. Pass an int for reproducible output across multiple function calls.
raw_treino_x, raw_teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,stratify = y) #Separando os dados para treino e teste e estipulando o tamanho para o teste, também. 
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x),len(teste_x)))

#Trinando o escalador - Treinando um novo processo de escala: Isso o ocorre devido a discrepância entre os eixos: em X, teremos valores de 0 a 100, e em Y de 0 a 30000. Esses algoritmos são muito suscetíveis a escala, e darão menos valor para variações menores, como é o caso de X.
scaler = StandardScaler() #Escalas distintas em features causam um desbalanço no algoritmo
scaler.fit(raw_treino_x)
treino_x = scaler.transform(raw_treino_x)
teste_x = scaler.transform(raw_teste_x)

modelo = DecisionTreeClassifier()
modelo.fit(treino_x, treino_y)

previsoes = modelo.predict(teste_x) #Predizendo através dos meus dados de teste, após ser treinado com meus dados de treino
taxa_de_acerto = accuracy_score(teste_y, previsoes)* 100 #Imprimir minha taxa de acerto ou "acurácia" - Comparação com os meus dados reais de teste
print("A acurácia foi %.2f" % taxa_de_acerto) 


dot_data = export_graphviz(modelo, out_file=None)
grafico = graphviz.Source(dot_data)
grafico

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 73.20
